In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset

import sklearn.model_selection as sk



In [ ]:
# Path for (unlabeled) test data
test_path = '/content/drive/MyDrive/FashionClassification/Test/'

In [ ]:
import re
def num_sort(test_string):
    return list(map(int, re.findall(r'\d+', test_string)))[0]

In [ ]:
# Prepare test data set

data=[]
label=[]

files = os.listdir(test_path)
files.sort(key=num_sort)

for f in files:
  img = Image.open(test_path+f)
  img = img.resize((224, 224)) #(150,150)
  one_img = np.asarray(np.float32(img))
  norm_img = (one_img-np.min(one_img))/(np.max(one_img)-np.min(one_img))
  data.append(norm_img.transpose(2,0,1))  # (150,150,3) --> (3,150,150)
  label.append(0)    # dummy label for preparing dataloader

        
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

test_X = torch.from_numpy(data)
test_Y = torch.from_numpy(label)

test_X = test_X.type(torch.cuda.FloatTensor)
test_Y = test_Y.type(torch.cuda.LongTensor)

test_dataset = TensorDataset(test_X,test_Y)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
print(test_X.shape, test_Y.shape)

torch.Size([6, 3, 224, 224]) torch.Size([6])


In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(DEVICE)

cuda


In [ ]:
# Build convolutional neural net
# IMPORTNT: This structure should be same as the one used for training (so simply copy and paste the correponding codes)
##### To do #####
# 1x1 convolution
def conv_1(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU(),
    )
    return model

# 1x1 conv -> 3x3 convolution
def conv_1_3(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,kernel_size = 3,stride = 1,padding = 1),
        nn.ReLU()
    )
    return model

# 1x1 conv -> 5x5 convolution
def conv_1_5(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,5,1,2),
        nn.ReLU()
    )
    return model

# 3x3 maxPooling -> 1x1 convolution
def max_3_1(in_dim, out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        nn.Conv2d(in_dim, out_dim, 1,1),
        nn.ReLU(),
    )
    return model


#Inception Module
class inception_module(nn.Module):
    def __init__(self,in_dim,out_dim_1,mid_dim_3,out_dim_3,mid_dim_5,out_dim_5,pool_dim):
        super(inception_module,self).__init__()
        # 1x1 Convolution
        self.conv_1 = conv_1(in_dim,out_dim_1)
        
        # 1x1 Convolution -> 3x3 Convolution
        self.conv_1_3 = conv_1_3(in_dim,mid_dim_3,out_dim_3)
        
        # 1x1 Convolution -> 5x5 Convolution
        self.conv_1_5 = conv_1_5(in_dim,mid_dim_5,out_dim_5)
        
        # 3x3 MaxPooling -> 1x1 Convolution
        self.max_3_1 = max_3_1(in_dim,pool_dim)

    def forward(self,x):
        out_1 = self.conv_1(x)
        out_2 = self.conv_1_3(x)
        out_3 = self.conv_1_5(x)
        out_4 = self.max_3_1(x)
        # concat
        output = torch.cat([out_1,out_2,out_3,out_4],1)
        return output


In [ ]:
# GoogleNet
class GoogLeNet(nn.Module):
    def __init__(self, base_dim=64, num_classes=3):
        super(GoogLeNet, self).__init__()
        self.num_classes=num_classes
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels= base_dim, kernel_size=7, stride = 2, padding = 3),
            nn.MaxPool2d(3,2,1),
            nn.Conv2d(in_channels = base_dim, out_channels=base_dim*3, kernel_size=3,stride =1,padding =1),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            inception_module(base_dim*3,64,96,128,16,32,32),
            inception_module(base_dim*4,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_3 = nn.Sequential(
            inception_module(480,192,96,208,16,48,64),
            inception_module(512,160,112,224,24,64,64),
            inception_module(512,128,128,256,24,64,64),
            inception_module(512,112,144,288,32,64,64),
            inception_module(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_4 = nn.Sequential(
            inception_module(832,256,160,320,32,128,128),
            inception_module(832,384,192,384,48,128,128), # 7x7x1024
            nn.AvgPool2d(7,1), # 1x1x1024
        )
        self.layer_5 = nn.Dropout2d(0.4)
        self.fc_layer = nn.Linear(1024,self.num_classes)
                
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = out.view(out.shape[0],-1) 
        out = self.fc_layer(out)
        return out
  
model = GoogLeNet().to(DEVICE)  # 모델 GPU로
model # Print network

GoogLeNet(
  (layer_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): inception_module(
      (conv_1): Sequential(
        (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (conv_1_3): Sequential(
        (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
      (conv_1_5): Sequential(
        (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (3): ReLU()
      )
      (max_3_1): Sequential

In [ ]:
# load trained model
##### To do #####

model = torch.load('/content/drive/MyDrive/FashionClassification/Model_TeamB')

In [ ]:
# evaluation
model.eval()
f_pred = []

with torch.no_grad():
  for inputs, labels in test_loader:
    inputs, labels = Variable(inputs), Variable(labels)
    output = model(inputs)

    idx, pred = output.max(1)
    pred = (pred.detach().cpu().numpy())

    f_pred.extend(pred.tolist())

In [ ]:
# converting 0/1/2 label value into Resort/Retro/Sporty

for index, value in enumerate(f_pred):
  if value==0:
    f_pred[index]='Resort'
  elif value==1:
    f_pred[index]='Retro'
  else:
    f_pred[index]='Sporty'

In [ ]:
# Submission file
img_files = [i for i in range(len(f_pred))]
submission = pd.DataFrame(index = range(0,len(f_pred)), columns = {'img','class'})
submission['img'] = img_files
submission['class'] = f_pred


In [ ]:
submission

,class,img
0,Retro,0
1,Retro,1
2,Retro,2
3,Retro,3
4,Retro,4
5,Retro,5


In [ ]:
submission.to_csv('/content/drive/MyDrive/FashionClassification/TeamB_result.csv',index=True)